In [3]:
from experimental_data_digitalization.params import *
from pypdf import PdfReader
import pymupdf
import pandas as pd

In [4]:

pdfs = [file for file in LOCAL_DATA_PATH.iterdir() if file.suffix == '.pdf']
pdfs


[PosixPath('/home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/jacs.8b01774.pdf'),
 PosixPath('/home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771-s1.pdf'),
 PosixPath('/home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/ja8b01774_si_001.pdf'),
 PosixPath('/home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/24gpt4v.pdf'),
 PosixPath('/home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/d3dd00239j1.pdf'),
 PosixPath('/home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf')]

# extract text (PyPDF)

In [26]:
reader = PdfReader(pdfs[3])

page = reader.pages[0]
page.extract_text()

"Image and data mining in reticular chemistry\npowered by GPT-4V †\nZhiling Zheng,abcZhiguo He,abOmar Khattab,dNakul Rampal,abcMatei A. Zaharia,e\nChristian Borgs,ceJennifer T. Chayescefghand Omar M. Yaghi *abci\nThe integration of arti ﬁcial intelligence into scienti ﬁc research opens new avenues with the advent of GPT-\n4V, a large language model equipped with vision capabilities. In this study, we demonstrate that GPT-4V,accessible through the ChatGPT web user interface or an API, o ﬀers promising possibilities in navigating\nand mining complex data for metal –organic frameworks (MOFs) especially from graphical sources ( e.g.\nsorption isotherms, powder X-ray di ﬀraction patterns, thermogravimetric analysis graphs, etc.). Our\napproach involved an automated process of converting 346 scholarly articles into 6240 images, whichrepresents a benchmark dataset in this task, followed by deploying GPT-4V to categorize and analyzethese images using natural language prompts, which can be writ

In [25]:
reader = PdfReader(pdfs[3]) # open a document
with open("output_pypdf.txt", "wb") as file: # create a text output
    for page in reader.pages: # iterate the document pages
        text = page.extract_text().encode("utf8") # get plain text (is in UTF-8)
        file.write(text) # write text of page
        file.write(bytes((12,))) # write page delimiter (form feed 0x0C)

# extract image (PyPDF)

In [29]:
reader = PdfReader(pdfs[3])

for page in reader.pages:
    for count, image_file_object in enumerate(page.images):
        with open(str(count) + image_file_object.name, "wb") as fp:
            fp.write(image_file_object.data)

# extract text (PyMuPDF)

In [21]:
doc = pymupdf.open(pdfs[3]) # open a document
with open("output_pymupdf.txt", "wb") as file: # create a text output
    for page in doc: # iterate the document pages
        text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
        file.write(text) # write text of page
        file.write(bytes((12,))) # write page delimiter (form feed 0x0C)

print(doc[0].get_text())

Image and data mining in reticular chemistry
powered by GPT-4V†
Zhiling Zheng,
abc Zhiguo He,ab Omar Khattab,d Nakul Rampal,abc Matei A. Zaharia,e
Christian Borgs,ce Jennifer T. Chayescefgh and Omar M. Yaghi
*abci
The integration of artiﬁcial intelligence into scientiﬁc research opens new avenues with the advent of GPT-
4V, a large language model equipped with vision capabilities. In this study, we demonstrate that GPT-4V,
accessible through the ChatGPT web user interface or an API, oﬀers promising possibilities in navigating
and mining complex data for metal–organic frameworks (MOFs) especially from graphical sources (e.g.
sorption isotherms, powder X-ray diﬀraction patterns, thermogravimetric analysis graphs, etc.). Our
approach involved an automated process of converting 346 scholarly articles into 6240 images, which
represents a benchmark dataset in this task, followed by deploying GPT-4V to categorize and analyze
these images using natural language prompts, which can be written by

# extract images (PyMuPDF)

In [30]:
for page_index in range(len(doc)): # iterate over pdf pages
    page = doc[page_index] # get the page
    image_list = page.get_images()

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index}")
    else:
        print("No images found on page", page_index)

    for image_index, img in enumerate(image_list, start=1): # enumerate the image list
        xref = img[0] # get the XREF of the image
        pix = pymupdf.Pixmap(doc, xref) # create a Pixmap

        if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
            pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

        pix.save("page_%s-image_%s.png" % (page_index, image_index)) # save the image as png
        pix = None

Found 1 images on page 0
Found 2 images on page 1
Found 1 images on page 2
Found 1 images on page 3
Found 2 images on page 4
Found 2 images on page 5
Found 2 images on page 6
Found 1 images on page 7
Found 1 images on page 8
Found 1 images on page 9
Found 1 images on page 10


# extract Vector Graphics (PyMuPDF)

In [99]:
doc = pymupdf.open(pdfs[0])
for number, page in enumerate(doc):
    paths = page.get_drawings()
    print(f"page-index {number}: {len(paths)} vector graphics")

page = doc[0]
paths = page.get_drawings()
paths[0].keys(), [path.get('fill', 'is None') for path in paths if path.get('fill')]

page-index 0: 52 vector graphics
page-index 1: 16 vector graphics
page-index 2: 16 vector graphics
page-index 3: 19 vector graphics


(dict_keys(['items', 'type', 'even_odd', 'fill_opacity', 'fill', 'rect', 'seqno', 'layer', 'closePath', 'color', 'width', 'lineCap', 'lineJoin', 'dashes', 'stroke_opacity']),
 [(1.0, 1.0, 1.0),
  (1.0, 1.0, 1.0),
  (0.5071183443069458, 0.49796292185783386, 0.6900892853736877),
  (0.5071183443069458, 0.49796292185783386, 0.6900892853736877),
  (0.15284962952136993, 0.19644464552402496, 0.49930572509765625),
  (1.0, 1.0, 1.0),
  (0.15284962952136993, 0.19644464552402496, 0.49930572509765625),
  (1.0, 1.0, 1.0),
  None,
  None,
  None])

In [102]:
page = doc[0]
paths = page.get_drawings()  # extract existing drawings
# this is a list of "paths", which can directly be drawn again using Shape
# -------------------------------------------------------------------------
#
# define some output page with the same dimensions
outpdf = pymupdf.open()
outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
shape = outpage.new_shape()  # make a drawing canvas for the output page
# --------------------------------------
# loop through the paths and draw them
# --------------------------------------
for path in paths:
    # ------------------------------------
    # draw each entry of the 'items' list
    # ------------------------------------

    for item in path["items"]:  # these are the draw commands
        if item[0] == "l":  # line
            shape.draw_line(item[1], item[2])
        elif item[0] == "re":  # rectangle
            shape.draw_rect(item[1])
        elif item[0] == "qu":  # quad
            shape.draw_quad(item[1])
        elif item[0] == "c":  # curve
            shape.draw_bezier(item[1], item[2], item[3], item[4])
        else:
            raise ValueError("unhandled drawing", item)
    # ------------------------------------------------------
    # all items are drawn, now apply the common properties
    # to finish the path
    # ------------------------------------------------------
    shape.finish(
        fill=path["fill"],  # fill color
        color=path["color"],  # line color
        dashes=path["dashes"],  # line dashing
        even_odd=path.get("even_odd", True),  # control color of overlaps
        closePath=path["closePath"],  # whether to connect last and first point
        lineJoin=path["lineJoin"],  # how line joins should look like
        lineCap=max(path["lineCap"]),  # how line ends should look like
        width=path["width"],  # line width
        stroke_opacity=path.get("stroke_opacity", 1),  # same value for both
        fill_opacity=path.get("fill_opacity", 1),  # opacity parameters
        )
# all paths processed - commit the shape to its page
shape.commit()
outpdf.save("drawings-page-0.pdf")

TypeError: 'NoneType' object is not iterable

# extract table

In [109]:
from pprint import pprint

doc = pymupdf.open(pdfs[5])
for idx, page in enumerate(doc):
    # page = doc[0] # get the 1st page of the document
    tabs = page.find_tables() # locate and extract any tables on page
    print(f"{len(tabs.tables)} found on {page}") # display number of found tables

    if tabs.tables:  # at least one table found?
     pprint(tabs[0].extract())  # print content of first table

0 found on page 0 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf
0 found on page 1 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf
0 found on page 2 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf
0 found on page 3 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf
0 found on page 4 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf
0 found on page 5 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2771.pdf
0 found on page 6 of /home/jerome-roeser/code/jerome-roeser/11-Personal-Projects/git_repos/Experimental-Data-Digitalization/data/nchem.2